# Importing dataset

In [294]:
import pandas as pd
import numpy as np
import random

import warnings
warnings.filterwarnings('ignore')

In [295]:
df = pd.read_csv('../app/data/train_users_2.csv', parse_dates=[1,3])
df

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaT,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaT,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213446,zxodksqpep,2014-06-30,20140630235636,NaT,MALE,32.0,basic,0,en,sem-brand,google,omg,Web,Mac Desktop,Safari,NDF
213447,mhewnxesx9,2014-06-30,20140630235719,NaT,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome,NDF
213448,6o3arsjbb4,2014-06-30,20140630235754,NaT,-unknown-,32.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,NDF
213449,jh95kwisub,2014-06-30,20140630235822,NaT,-unknown-,NaN,basic,25,en,other,other,tracked-other,iOS,iPhone,Mobile Safari,NDF


* First I am gonna delete the useless features that help use to decide if a user gonna book or just gonna lose his time so best for him to delete the app.

In [296]:
df = df.drop(['id', 'timestamp_first_active', 'date_first_booking', 'signup_flow', 'affiliate_channel'], axis=1)

In [297]:
df = df.dropna(subset=['age'])

In [298]:
"""
for i in ['gender', 'signup_method', 'language', 'affiliate_provider', 'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser', 'country_destination']:
    print(df[i].value_counts(normalize=True))
    print('********************')
    
for i in ['gender', 'signup_method', 'language', 'affiliate_provider', 'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser', 'country_destination']:
    print(new_df[i].value_counts(normalize=True))
    print('********************')
    
"""
print('')

After deleting the users that have nan value in age and checking if it is any change of diversity of the other features, it is almost the same. 

# Numerical features

In [299]:
df[df.select_dtypes('datetime').columns]

,date_account_created
1,2011-05-25
2,2010-09-28
3,2011-12-05
4,2010-09-14
6,2010-01-02
...,...
213441,2014-06-30
213443,2014-06-30
213445,2014-06-30
213446,2014-06-30


In [300]:
df['year_account_created'] = df.date_account_created.dt.year
df['month_account_created'] = df.date_account_created.dt.month

df = df.drop('date_account_created', axis=1)

In [301]:
num_cols = df.select_dtypes('number').columns

df[num_cols].isna().sum()

age                      0
year_account_created     0
month_account_created    0
dtype: int64

In [302]:
df[df.select_dtypes('number').columns]

,age,year_account_created,month_account_created
1,38.0,2011,5
2,56.0,2010,9
3,42.0,2011,12
4,41.0,2010,9
6,46.0,2010,1
...,...,...,...
213441,34.0,2014,6
213443,36.0,2014,6
213445,23.0,2014,6
213446,32.0,2014,6


In [303]:
df[num_cols].corr().style.background_gradient(cmap='coolwarm')

,age,year_account_created,month_account_created
age,1.000000,-0.035580,0.001388
year_account_created,-0.035580,1.000000,-0.437561
month_account_created,0.001388,-0.437561,1.000000


# Object features:

In [304]:
# Feature selection (categorical features)
cat_cols = df.select_dtypes('object').columns.tolist()

cat_cols.remove('country_destination')

In [305]:
df[cat_cols].isna().sum()

gender                        0
signup_method                 0
language                      0
affiliate_provider            0
first_affiliate_tracked    2032
signup_app                    0
first_device_type             0
first_browser                 0
dtype: int64

***Affiliate tracked***

In [306]:
df['first_affiliate_tracked'] = new_df.first_affiliate_tracked.fillna(new_df.first_affiliate_tracked.mode().iloc[0])

***Gender***

In [307]:
df.gender.value_counts(normalize=True), df.gender.value_counts()

(gender
 FEMALE       0.460007
 MALE         0.403926
 -unknown-    0.134249
 OTHER        0.001817
 Name: proportion, dtype: float64,
 gender
 FEMALE       57713
 MALE         50677
 -unknown-    16843
 OTHER          228
 Name: count, dtype: int64)

In [308]:
"""
df.loc[df.gender.isin(['OTHER','-unknown-']), 'gender'] = np.nan
proba = df.gender.value_counts(normalize=True)
rep = np.random.choice(['FEMALE', 'MALE'], df.shape[0], p = proba)
df['gender'] = df.gender.fillna(pd.Series(rep))
"""
df.loc[df.gender.isin(['OTHER','-unknown-']), 'gender'] = np.nan
df = df.dropna(subset=['gender'])

In [309]:
df[cat_cols]

,gender,signup_method,language,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser
1,MALE,facebook,en,google,untracked,Web,Mac Desktop,Chrome
2,FEMALE,basic,en,direct,untracked,Web,Windows Desktop,IE
3,FEMALE,facebook,en,direct,untracked,Web,Mac Desktop,Firefox
6,FEMALE,basic,en,craigslist,untracked,Web,Mac Desktop,Safari
7,FEMALE,basic,en,direct,omg,Web,Mac Desktop,Safari
...,...,...,...,...,...,...,...,...
213440,FEMALE,basic,en,direct,untracked,iOS,iPhone,Mobile Safari
213441,FEMALE,basic,en,direct,linked,Web,Mac Desktop,Chrome
213443,FEMALE,basic,en,google,linked,Web,Mac Desktop,Safari
213445,FEMALE,basic,en,google,omg,Web,Windows Desktop,IE


# Correct data:

In [311]:
df.columns.tolist()

['gender',
 'age',
 'signup_method',
 'language',
 'affiliate_provider',
 'first_affiliate_tracked',
 'signup_app',
 'first_device_type',
 'first_browser',
 'country_destination',
 'year_account_created',
 'month_account_created']

**SignUp method**

In [312]:
df.signup_method.value_counts()

signup_method
facebook    56851
basic       51445
google         94
Name: count, dtype: int64

**Language**

In [313]:
df.language.value_counts()

language
en    104218
zh       821
fr       760
es       589
ko       374
de       364
it       349
ru       262
pt       168
ja       123
sv        75
nl        46
pl        41
tr        40
da        30
el        21
cs        19
no        18
th        18
id        17
hu        16
fi        11
ca         4
is         4
hr         2
Name: count, dtype: int64

In [314]:
rep_value_language = {
    'en': 'english',
    'zh': 'chinese',
    'fr': 'french',
    'es': 'spanish',
    'de': 'german',
    'ko': 'korean',
    'it': 'italian',
    'ru': 'russian',
    'pt': 'other',
    'ja': 'other',
    'sv': 'other',
    'nl': 'other',
    'tr': 'other',
    'pl': 'other',
    'da': 'other',
    'el': 'other',
    'no': 'other',
    'cs': 'other',
    'th': 'other',
    'id': 'other',
    'hu': 'other',
    'fi': 'other',
    'ca': 'other',
    'is': 'other',
    'hr': 'other'
}
df['language'] = df.language.replace(rep_value_language)

**Affiliate provider**

In [315]:
# where the marketing is e.g. google, craigslist, other

df.affiliate_provider.value_counts()

affiliate_provider
direct                 68792
google                 25624
other                   7743
craigslist              1896
facebook                1499
bing                    1059
padmapper                410
facebook-open-graph      389
vast                     335
yahoo                    225
gsp                      172
meetup                   126
email-marketing           67
naver                     23
yandex                    14
baidu                     13
wayn                       3
Name: count, dtype: int64

In [316]:
rep_value_affiliate_provider = {
    'craigslist': 'other',
    'padmapper': 'other',
    'facebook-open-graph': 'facebook',
    'vast': 'other',
    'gsp': 'other',
    'meetup': 'other',
    'email-marketing': 'other',
    'naver': 'other',
    'baidu': 'other',
    'yandex': 'other',
    'wayn': 'other'
}

df['affiliate_provider'] = df.affiliate_provider.replace(rep_value_affiliate_provider)

**First affiliate tracked**

In [317]:
# whats the first marketing the user interacted with before the signing up

df.first_affiliate_tracked.value_counts()

first_affiliate_tracked
untracked        57604
linked           24730
omg              21751
tracked-other     3469
product            720
marketing           91
local ops           25
Name: count, dtype: int64

Here I'm going to convert all values except untracked to marketing, because all the tracking affiliates are from marketing

In [318]:
rep_value_first_affiliate_tracked = {
    'linked': 'ads',
    'omg': 'ads',
    'tracked-other': 'ads',
    'local ops': 'local business'
}

df['first_affiliate_tracked'] = df.first_affiliate_tracked.replace(rep_value_first_affiliate_tracked)

**SignUp App**

In [319]:
df.signup_app.value_counts()

signup_app
Web        95712
iOS         8747
Android     1972
Moweb       1959
Name: count, dtype: int64

**First device type**

In [320]:
df.first_device_type.value_counts()

first_device_type
Mac Desktop           50481
Windows Desktop       36295
iPhone                 9126
iPad                   6624
Other/Unknown          3575
Android Phone          1078
Desktop (Other)         644
Android Tablet          530
SmartPhone (Other)       37
Name: count, dtype: int64

In [321]:
rep_value_device_type = {
    'Mac Desktop': 'MacBook',
    'Windows Desktop': 'Laptop',
    'Other/Unknown': 'Android Phone',
    'Desktop (Other)': 'Laptop',
    'SmartPhone (Other)': 'Android Phone',
    'Other': 'Android Phone'
}

df['first_device_type'] = df.first_device_type.replace(rep_value_device_type)

**First browser**

In [322]:
df.first_browser.value_counts()

first_browser
Chrome                36478
Safari                24145
Firefox               17751
-unknown-             10620
IE                     9079
Mobile Safari          8957
Chrome Mobile           492
Android Browser         363
Opera                    99
AOL Explorer             89
Silk                     69
Chromium                 42
BlackBerry Browser       34
Maxthon                  25
RockMelt                 20
Apple Mail               19
Mobile Firefox           14
Sogou Explorer           12
IceWeasel                10
Iron                      9
Yandex.Browser            9
IE Mobile                 9
Pale Moon                 6
SiteKiosk                 6
CoolNovo                  4
Camino                    4
Avant Browser             4
TenFourFox                3
TheWorld Browser          2
Opera Mobile              2
Opera Mini                2
Mozilla                   2
wOSBrowser                2
NetNewsWire               1
PS Vita browser           1
SeaMon

In [323]:
rep_value_browser = {
    'Chrome': 'Google Chrome',
    '-unknown-': 'Other Browser',
    'IE': 'Microsoft Edge',
    'Mobile Safari': 'Safari',
    'Chrome Mobile': 'Google Chrome',
    'Android Browser': 'Other Browser',
    'AOL Explorer': 'Other Browser',
    'Silk': 'Other Browser',
    'Chromium': 'Google Chrome',
    'BlackBerry Browser': 'Other Browser',
    'Maxthon': 'Other Browser',
    'Apple Mail': 'Safari',
    'RockMel': 'Other Browser',
    'Mobile Firefox': 'Firefox',
    'Sogou Explorer': 'Other Browser',
    'IE Mobile': 'Microsoft Edge',
    'SiteKiosk': 'Other Browser',
    'Iron': 'Other Browser',
    'IceWeasel': 'Other Browser',
    'Yandex.Browser': 'Other Browser',
    'Pale Moon': 'Other Browser',
    'Camino': 'Other Browser',
    'CoolNovo': 'Other Browser',
    'Avant Browser': 'Other Browser',
    'SeaMonkey': 'Other Browser',
    'SlimBrowser': 'Other Browser',
    'TheWorld Browser': 'Other Browser',
    'Opera Mobile': 'Opera',
    'Mozilla': 'Firefox',
    'wOSBrowser': 'Other Browser',
    'Opera Mini': 'Other Browser',
    'NetNewsWire': 'Other Browser',
    'PS Vita browser': 'Other Browser',
    'CometBird': 'Other Browser',
    'Kindle Browser': 'Other Browser',
    'Comodo Dragon': 'Other Browser',
    'Stainless': 'Other Browser',
    'TenFourFox': 'Other Browser',
    'RockMelt': 'Other Browser'
}

df['first_browser'] = df.first_browser.replace(rep_value_browser)

**Country destination**

In [324]:
df.country_destination.value_counts()

country_destination
NDF      52057
US       39866
other     6212
FR        3070
IT        1671
GB        1479
ES        1432
CA         874
DE         726
NL         498
AU         371
PT         134
Name: count, dtype: int64

In [329]:
rep_value_destination = {
    'NDF': 'LoseTime',      
    'US': 'Booked',   
    'other': 'Booked',
    'FR': 'Booked',
    'IT': 'Booked',
    'GB': 'Booked',
    'ES': 'Booked',
    'CA': 'Booked',
    'DE': 'Booked',
    'NL': 'Booked',
    'AU': 'Booked',
    'PT': 'Booked'
}

df['country_destination'] = df.country_destination.replace(rep_value_destination)

df.rename(columns={'country_destination': 'TARGET'}, inplace=True)

In [330]:
df

,gender,age,signup_method,language,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,TARGET,year_account_created,month_account_created
1,MALE,38.0,facebook,english,google,untracked,Web,MacBook,Google Chrome,LoseTime,2011,5
2,FEMALE,56.0,basic,english,direct,untracked,Web,Laptop,Microsoft Edge,Booked,2010,9
3,FEMALE,42.0,facebook,english,direct,untracked,Web,MacBook,Firefox,Booked,2011,12
6,FEMALE,46.0,basic,english,other,untracked,Web,MacBook,Safari,Booked,2010,1
7,FEMALE,47.0,basic,english,direct,ads,Web,MacBook,Safari,Booked,2010,1
...,...,...,...,...,...,...,...,...,...,...,...,...
213440,FEMALE,24.0,basic,english,direct,untracked,iOS,iPhone,Safari,LoseTime,2014,6
213441,FEMALE,34.0,basic,english,direct,ads,Web,MacBook,Google Chrome,Booked,2014,6
213443,FEMALE,36.0,basic,english,google,ads,Web,MacBook,Safari,Booked,2014,6
213445,FEMALE,23.0,basic,english,google,ads,Web,Laptop,Microsoft Edge,Booked,2014,6


In [332]:
df.to_csv('users_data_cleaned.csv', index=False)